**Install python dependencies**

In [ ]:
!pip install -q -r ./dependencies/requirements.txt

**Load python libraries**

In [2]:
import pandas as pd
from tqdm import tqdm
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, cross_val_predict, cross_validate, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier
from sklearn import tree

import matplotlib.pyplot as plt
import seaborn as sns

**Load Data**

In [39]:
df = pd.read_csv('./data/final_scores.csv')

odds_columns = ['odds_win', 'odds_draw', 'odds_lose']

df_prev_score = pd.DataFrame()

df['club_id'] = LabelEncoder().fit_transform(df['club_id'])
df['position'] = LabelEncoder().fit_transform(df['position'])
df['name'] = LabelEncoder().fit_transform(df['name'])

for player, df_player in df.groupby(['name']):
    
    df_player = df_player.sort_values('matchday')
    df_player.set_index('matchday', inplace=True)
    df_player['prev_score'] = df_player.final_score.ewm(alpha=0.5, adjust=False).mean().map(lambda x: int(x)).shift(periods=1, fill_value=0)

    df_prev_score = df_prev_score.append(df_player)

df = df_prev_score

df_one_hot = pd.get_dummies(df['club_id'])

df = df.drop('club_id', axis=1)

df.merge(df_one_hot, right_index=True)

# df = df.join(df_one_hot)

# df

# df = df.drop(['name', 'position', 'odds_win', 'odds_draw', 'odds_lose'], axis=1)

# y = df['final_score']
# X = df.drop('final_score', axis='columns')

# X

# prev_score, is_home, club (one hot), odds, position (one hot) ---> magic ---> final_score

MergeError: Must pass left_on or left_index=True

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)

In [ ]:
plt.figure(figsize=(12,10))
sns.heatmap(X_train.corr(), annot=True, cmap=plt.cm.Reds)
plt.show()

In [ ]:
y_train.value_counts(normalize=True)

In [ ]:
dt = tree.DecisionTreeRegressor()
dt_cv_score = cross_val_score(dt, X_train, y_train, cv=StratifiedKFold(shuffle=True))
print(dt_cv_score)
print(dt_cv_score.mean())